# Fraud Detection Ledger 프로젝트 개요
본 프로젝트는 ERP 회계전표 데이터를 활용하여 자금 횡령 및 부정거래를 자동으로 탐지하는 시스템입니다.
데이터 불러오기, 전처리, 비지도학습, 지도학습, 시각화를 포함하였습니다.

In [ ]:
import pandas as pd

#데이터 불러오기
df = pd.read_csv(r'C:\Users\HyunSu\Desktop\Fraud Detection Ledger\data\fake_journal_entries_500.csv')

#데이터 정보 확인
print(df.head())
print(df.info())
print(df.describe())

In [ ]:
#머신러닝 준비과정

# 시간 파생 변수 및 야간 입력 여부
df['date'] = pd.to_datetime(df['date'])
df['time'] = pd.to_datetime(df['time'], format='%H:%M')
df['hour'] = df['time'].dt.hour
df['is_late_entry'] = (df['hour'] >= 18).astype(int)


# 공휴일 학습
import holidays
kr_holidays = holidays.KR(years=2025)
kr_holiday_dates = pd.to_datetime(list(kr_holidays.keys()))
df['is_holiday'] = df['date'].isin(kr_holiday_dates).astype(int)


In [ ]:
#비지도학습 (isoltation forest)

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest

# isolation forest model 적용
X_iso = df[['amount', 'hour', 'is_late_entry', 'is_holiday']]
iso_model = IsolationForest(contamination=0.1, random_state=57)
df['anomaly_score'] = iso_model.fit_predict(X_iso)  # -1: 이상, 1: 정상
df['is_anomaly'] = df['anomaly_score'] == -1

In [ ]:
# 지도학습 (Random Forest)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 사용자별 반복 고액 이체 여부 확인
df['high_amount'] = df['amount'].abs() > 1_000_000
user_high_transfer_count = df.groupby('entered_by')['high_amount'].transform('sum')
df['user_high_transfer_count'] = user_high_transfer_count

# 이상 조건 정의 (조건이 2개 이상 충족 시 이상으로 간주)
conditions = [
    df['amount'].abs() > 1_000_000,
    df['is_holiday'] == 1,
    df['is_late_entry'] == 1,
    df['user_high_transfer_count'] >= 3,
    df['vendor'].str.contains('Ltd'),
]
df['potential_fraud_complex'] = (sum(conditions) >= 2).astype(int)

# 모델 학습 및 평가
X = df[['amount', 'hour', 'is_late_entry', 'is_holiday']]
y = df['potential_fraud_complex']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=57)

clf = RandomForestClassifier(random_state=57)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
df['rf_pred'] = clf.predict(X)

print("Random Forest 평가 결과:")
print(classification_report(y_test, y_pred))


# 비지도학습 평가용 데이터 분할 (Isolation Forest → 지도 학습 평가 방식 적)
from sklearn.ensemble import IsolationForest

X_iso = df[['amount', 'hour', 'is_late_entry', 'is_holiday']]
y_iso = df['is_anomaly'].astype(int)

X_train_iso, X_test_iso, y_train_iso, y_test_iso = train_test_split(
    X_iso, y_iso, test_size=0.3, stratify=y_iso, random_state=57
)

iso = IsolationForest(contamination=0.1, random_state=57)
iso.fit(X_train_iso)

y_pred_iso = (iso.predict(X_test_iso) == -1).astype(int)

print("Isolation Forest 평가 결과:")
print(classification_report(y_test_iso, y_pred_iso))

In [ ]:
#지도학습과 비지도학습 결과 비교
print("비지도학습 탐지 결과")
print(df['is_anomaly'].value_counts())

print("지도학습 라벨링 기준 탐지 결과")
print(df['potential_fraud_complex'].value_counts())

print("교차표로 비교 (얼마나 겹치는지)")
print(pd.crosstab(df['is_anomaly'], df['potential_fraud_complex']))


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# 모델 정의
rf_model = RandomForestClassifier(random_state=57)

# 교차 검증 수행 (5-겹 교차 검증)
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5)

# 결과 출력
print(f"교차 검증 정확도: {cv_scores}")
print(f"평균 정확도: {cv_scores.mean():.4f}")
print(f"정확도 표준편차: {cv_scores.std():.4f}")


In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

# 글꼴설정
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False


In [ ]:
# 교차 검증 정확도 (예시 데이터)
cv_scores = [0.93333333, 0.88, 0.89333333, 0.92, 0.89333333]

# x축: fold 번호
folds = np.arange(1, len(cv_scores) + 1)

# 시각화
plt.figure(figsize=(8, 6))
plt.plot(folds, cv_scores, marker='o', linestyle='-', color='b', label='정확도')
plt.fill_between(folds, np.mean(cv_scores) - np.std(cv_scores), np.mean(cv_scores) + np.std(cv_scores),
                 color='b', alpha=0.2, label='정확도 표준편차 범위')
plt.axhline(np.mean(cv_scores), color='r', linestyle='--', label=f'평균 정확도: {np.mean(cv_scores):.4f}')
plt.title('교차 검증 정확도 시각화')
plt.xlabel('Fold 번호')
plt.ylabel('정확도')
plt.xticks(folds)
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# 금액 분포 차트 시각화
sns.set(style='whitegrid')
plt.figure(figsize=(10,6))
sns.histplot(df['amount'], bins=30, kde=True)
plt.title('금액 분포', fontproperties=font_prop)
plt.xlabel('금액', fontproperties=font_prop)
plt.ylabel('빈도', fontproperties=font_prop)
plt.show()

In [ ]:
# 시간대별 이상/정상 건수 집계
hourly = df.groupby(['hour', 'is_anomaly']).size().reset_index(name='count')
hourly_pivot = hourly.pivot(index='hour', columns='is_anomaly', values='count').fillna(0)
hourly_pivot.columns = ['정상', '이상']
mpl.rcParams['font.family'] = ['Malgun Gothic', 'sans-serif']
mpl.rcParams['axes.unicode_minus'] = False

# 비율 계산
hourly_pivot['이상비율'] = hourly_pivot['이상'] / (hourly_pivot['정상'] + hourly_pivot['이상'])

plt.figure(figsize=(10, 5))
sns.lineplot(data=hourly_pivot, x=hourly_pivot.index, y='이상비율', marker='o')
plt.xlabel('시간대 (hour)')
plt.ylabel('이상 거래 비율')
plt.title('시간대별 이상 거래 비율 시각화')
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(
    data=df,
    x='is_anomaly', 
    y='amount', 
    hue='is_anomaly',        # palette 사용 시 필수!
    palette=['C0', 'C1'],    # 0 → C0, 1 → C1
    legend=False             # 범례 숨기기
)
plt.yscale('log')
plt.xlabel('이상 여부 (0: 정상, 1: 이상)')
plt.ylabel('거래 금액 (로그 스케일)')
plt.title('이상 vs 정상 거래 금액 분포 비교')
plt.show()


In [ ]:
# 공휴일 / 야간 입력별 평균 이상 비율 계산
pivot = df.groupby(['is_holiday', 'is_late_entry'])['is_anomaly'].mean().reset_index()
pivot = pivot.pivot(index='is_holiday', columns='is_late_entry', values='is_anomaly')

# 히트맵 그리기
plt.figure(figsize=(6, 4))
sns.heatmap(pivot, annot=True, fmt=".2f", cmap='Reds')
plt.xlabel('야간 입력 여부 (0=낮, 1=야간)')
plt.ylabel('공휴일 여부 (0=비공휴일, 1=공휴일)')
plt.title('공휴일/야간에 따른 이상 거래 비율')
plt.show()
